# Marine EOV Broker



In [ ]:
from marine_eov_broker import MarineRiBroker
import logging
import matplotlib.pyplot as plt

logger = logging.getLogger()
logger.setLevel(logging.INFO)
# logger.setLevel(logging.DEBUG)

print(MarineRiBroker.ERDDAP_OUTPUT_FORMATS)
print(MarineRiBroker.EOV_LIST)

## Start the broker

It will take some time (though it still needs improvements on performances). This is because the broker will :
* load vocabularies upon startup
* load erddap datasets metadata from all erddap servers


**Question :**
Do we want to work with all datasets on Erddap servers ? Or do we want to build a fixed list for them ?

In [ ]:
%%time
broker = MarineRiBroker.MarineBroker()

## Create a request to the broker :
The user must provide the EOVs, min/max date/lat/lon, output format desired.

When creating a query, the broker :
* first looks at every dataset to see if they match any eov requested by the user
* then checks if the datasets match the time/bbox requested by the user

In [ ]:
start_date = "2018-01-01"
end_date = "2018-01-15"
# North-east Atlantic Ocean
min_lon = -30
min_lat = 41
max_lon = 2
max_lat = 62

#logger.setLevel(logging.DEBUG)

In [ ]:
%%time
response = broker.submit_request(["EV_SALIN", "EV_OXY", "EV_SEATEMP", "EV_CO2", "EV_CHLA"], 
                                 start_date,
                                 end_date,
                                 min_lon,
                                 min_lat,
                                 max_lon,
                                 max_lat,
                                 "nc"
                                 )

![EOV Broker in action](images/marine_eov_broker_mechanism.png)

## Results

The interesting part !
The broker provides a BrokerResponse object. It contains the variable **queries** which is a Pandas DataFrame.

The pandas DataFrame contains all the global attributes, query URL and ErddapRequest object for each dataset found for the user request.

In [ ]:
response.queries

**Or just the list of datasets ID**

In [ ]:
response.get_datasets_list()

### Access a dataset with its dataset ID

In [ ]:
dataset_id = response.get_datasets_list()[0]
print(dataset_id)

### Get the description of the EOVs found variables in the dataset

In [ ]:
response.get_dataset_EOVs_list(dataset_id)

### Get the query URL for the dataset ID

In [ ]:
response.get_dataset_query_url(dataset_id)

### Execute a query & get the result as a Pandas DataFrame...

In [ ]:
df = response.dataset_to_pandas_dataframe(dataset_id)
df

### ... or an Xarray dataset

In [ ]:
ds = response.dataset_to_xarray(dataset_id)
ds

### Only retrieve a specific EOV :

In [ ]:
ds = response.dataset_to_pandas_dataframe(dataset_id, "EV_OXY")
ds

### Download a dataset as a NetCDF file

In [ ]:
response.dataset_to_file_download(dataset_id, "nc")